<a href="https://colab.research.google.com/github/HuyenNguyenHelen/CORD-19-KG/blob/master/QA-models/covid_19_kg_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preliminaries

In [1]:
# If using Google Colab run this cell 

# select tensorflow version for colab 
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf 

print('TensorFlow  version: {}'.format(tf.__version__))

# Get the GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow  version: 1.15.2
Found GPU at: /device:GPU:0


In [3]:
%%capture 
# Install AmpliGraph library
! pip install ampligraph

# Required to visualize embeddings with tensorboard projector, comment out if not required!
! pip install --user tensorboard

# Required to plot text on embedding clusters, comment out if not required!
! pip install --user git+https://github.com/Phlya/adjustText

In [4]:
# All imports used in this tutorial 
%tensorflow_version 1.x
import ampligraph
import numpy as np
import pandas as pd
import tensorflow as tf
from ampligraph.datasets import load_fb15k_237
from ampligraph.evaluation import train_test_split_no_unseen, evaluate_performance, mr_score, mrr_score, hits_at_n_score
from ampligraph.discovery import query_topn, discover_facts, find_clusters
from ampligraph.latent_features import TransE, ComplEx, HolE, DistMult, ConvE, ConvKB
from ampligraph.utils import save_model, restore_model

def display_aggregate_metrics(ranks):
    print('Mean Rank:', mr_score(ranks)) 
    print('Mean Reciprocal Rank:', mrr_score(ranks)) 
    print('Hits@1:', hits_at_n_score(ranks, 1))
    print('Hits@10:', hits_at_n_score(ranks, 10))
    print('Hits@100:', hits_at_n_score(ranks, 100))

print('Ampligraph version: {}'.format(ampligraph.__version__))

Ampligraph version: 1.4.0


In [5]:
from ampligraph.datasets import load_fb15k_237, load_wn18rr, load_yago3_10
import pandas as pd
import requests
import io

## Loading Data 
(a triples dataset)

In [18]:
# Downloading the csv file from your GitHub account
re_pdf = requests.get('https://raw.githubusercontent.com/HuyenNguyenHelen/CORD-19-KG/master/Data/new_triples_with_predefined_relations_pdf_June20_.csv').content
re_pmc = requests.get('https://raw.githubusercontent.com/HuyenNguyenHelen/CORD-19-KG/master/Data/new_triples_with_predefined_relations_pmc_June20_.csv').content
pdf = pd.read_csv(io.StringIO(re_pdf.decode('utf-8')))
pmc =  pd.read_csv(io.StringIO(re_pmc.decode('utf-8')))
dataset = pd.concat([pmc, pdf])
print(dataset.columns)
dataset.drop(columns='Unnamed: 0', inplace=True)
dataset = dataset[['subject','new_relation', 'object' ]]
dataset.head(5)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'subject', 'object', 'new_relation'], dtype='object')


,subject,new_relation,object
0,dysfunction,disease_species,child
1,et dysfunction,disease_species,child
2,et dysfunction,disease_species,child
3,rsv,disease_disease,virus
4,treatment,treat_procedure_species,child


In [7]:
print('Total triples in the KG:', dataset.shape)

Total triples in the KG: (230330, 3)


In [8]:
print('the number of relations: \n', dataset['new_relation'].value_counts())

the number of relations: 
 disease_disease                    63192
gene_gene                          24172
disease_species                    22370
gene_disease                       14220
disease_gene                       13347
disease_treat_procedure            10184
treat_procedure_disease             8223
drug_disease                        7250
gene_treat_procedure                6142
treat_procedure_treat_procedure     5625
gene_species                        5298
drug_gene                           5270
drug_drug                           4613
treat_procedure_gene                4602
disease_symptom                     4354
treat_procedure_species             4263
disease_drug                        4044
drug_species                        3680
gene_drug                           3603
symptom_disease                     3411
symptom_symptom                     3266
drug_treat_procedure                3229
treat_procedure_drug                2578
symptom_species               

# Data Processing
Partition the triples into 10 portions that will be used for buiding QA systems

In [19]:
# Shuffle the dataset
dataset = dataset.sample(frac=1, axis=1).reset_index(drop=True)
subsets = np.array_split(dataset, 10)  
len(subsets)


10

# Developing QA system

## Developing model

### Create training, validation and test splits

In [20]:
from ampligraph.evaluation import train_test_split_no_unseen
def train_set_split (ds, test_frac = 1000, val_frac = 500 ):
  # get the validation set of size 500
  test_train, X_test = train_test_split_no_unseen(ds.values, test_frac, seed=0)

  # get the test set of size 1000 from the remaining triples
  X_train, X_val = train_test_split_no_unseen(test_train, val_frac, seed=0)

  print('Total triples:', dataset.shape)
  print('Size of train:', X_train.shape)
  print('Size of valid:', X_val.shape)
  print('Size of test:', X_test.shape)
  return X_train, X_val, X_test

### Model Training

In [21]:
from ampligraph.latent_features import TransE

def training (train_ds, val_ds, test_ds):
  # Train a KGE model
  model = TransE(k=300, 
                epochs=100, 
                eta=1, 
                loss='multiclass_nll', 
                initializer='xavier', initializer_params={'uniform': False},
                regularizer='LP', regularizer_params= {'lambda': 0.001, 'p': 3},
                optimizer= 'adam', optimizer_params= {'lr': 0.0001}, 
                seed= 0, batches_count= 100, verbose=True)

  model.fit(train_ds)
  # ----------------------
  # Evaluate: 
  # Filtered evaluation with ranking strategy assigning worst rank to break ties
  from ampligraph.utils import save_model, restore_model
  save_model(model, 'TransE.pkl')
  model = restore_model('TransE.pkl')

  # create the filter 
  X_filter = np.concatenate([train_ds, val_ds, test_ds], 0)

  # compute ranks
  ranks = evaluate_performance(test_ds, 
                              model=model, 
                              filter_triples=X_filter)

  # ranks are computed per triple
  print('Test set:', X_test.shape)
  print('Size of ranks:', ranks.shape)

  # Aggregate metrics show the aggregate performance of the model on the test set using a single number
  display_aggregate_metrics(ranks)
  # ----------------------
  
  return display_aggregate_metrics(ranks)

X_train, X_val, X_test = train_set_split(dataset)
training (X_train, X_val, X_test) #dataset.to_numpy()

Total triples: (230330, 3)
Size of train: (228830, 3)
Size of valid: (500, 3)
Size of test: (1000, 3)


100%|██████████| 1000/1000 [00:11<00:00, 89.27it/s]


Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 152.823
Mean Reciprocal Rank: 0.3711973943166797
Hits@1: 0.273
Hits@10: 0.559
Hits@100: 0.8015
Mean Rank: 152.823
Mean Reciprocal Rank: 0.3711973943166797
Hits@1: 0.273
Hits@10: 0.559
Hits@100: 0.8015


## Knowledge Discovery

In [23]:
from ampligraph.discovery import query_topn
def QA_retrieve (input, top_n ):
  head, relation1, tail = input[0], input[1], input[2]
  if len(relation1.split('_')) <= 2:
    relation2 = '{}_{}'.format(relation1.split('_')[1], relation1.split('_')[0])
  elif len(relation1.split('_')) == 3:
     relation2 = '{}_{}_{}'.format(relation1.split('_')[1], relation1.split('_')[2], relation1.split('_')[0])
  else:
    raise ValueError('error: relation length issue')
  # restore the previously saved model to save time
  model = restore_model('TransE.pkl')
  triples1, scores1 = query_topn(model, top_n=top_n, 
                              head=head, 
                              relation=relation1, 
                              tail=tail, 
                              ents_to_consider=None, 
                              rels_to_consider=None)
  triples2, scores2 = query_topn(model, top_n=top_n, 
                              head=tail, 
                              relation=relation2, 
                              tail=head, 
                              ents_to_consider=None, 
                              rels_to_consider=None)
  if len(triples1) + len(triples2) == top_n:
    triples = triples1.tolist() + triples2.tolist()
    scores = scores1.tolist() + scores2.tolist()
    # triples = np.concatenate((triples1, triples2), axis=0) 
    # scores = np.concatenate((scores1, scores2), axis=0)
  else:
    end_idx = round(top_n/2)
    triples = triples1.tolist()[:end_idx] + triples2.tolist()[:end_idx]
    scores = scores1.tolist()[:end_idx] + scores2.tolist()[:end_idx]
    # triples = np.concatenate((triples1[:(round(top_n/2))], triples2 [:(round(top_n/2))]), axis=0) 
    # scores = np.concatenate((scores1[:(round(top_n/2))] + scores2[:(round(top_n/2))]), axis=0) 
  
  for triple, score in zip(triples, scores):
      print('Score: {} \t {} '.format(score, triple))
  return triples, scores
 
# Query1: what are the possible therapeutic procedure of covid-19??
print ('------------------------------------------------------------')
print('what are the possible therapeutic procedure of covid-19?')
question1 = QA_retrieve (['covid-19', 'disease_treat_procedure', None], top_n = 20)

# Query2: What are related diseases of Covid-19
print ('------------------------------------------------------------')
print('What are related diseases of Covid-19')
question2 = QA_retrieve (['covid-19', 'disease_disease', None], top_n = 20)

# Query3: What are the genes/proteins related to Covid-19
print ('------------------------------------------------------------')
print('What are the genes/proteins related to Covid-19')
question3 = QA_retrieve (['covid-19', 'disease_gene', None], top_n = 20)

# Query4: What species transmit the covid-19
print ('------------------------------------------------------------')
print('What species transmit the covid-19?')
question4 = QA_retrieve (['covid-19', 'disease_species', None], top_n = 20)

# Query: what are symptoms of the covid-19
print ('------------------------------------------------------------')
print('what are symptoms of the covid-19?')
question5 = QA_retrieve (['covid-19', 'disease_symptom', None], top_n = 20)

# What are potential drugs to treat COVID-19?
print ('------------------------------------------------------------')
print('What are potential drugs to treat COVID-19?')
question6 = QA_retrieve (['covid-19', 'disease_drug', None], top_n = 20)

------------------------------------------------------------
what are the possible therapeutic procedure of covid-19?
Score: -14.424173355102539 	 ['covid-19', 'disease_treat_procedure', 'detected'] 
Score: -14.501384735107422 	 ['covid-19', 'disease_treat_procedure', 'presence'] 
Score: -14.54278564453125 	 ['covid-19', 'disease_treat_procedure', 'extracted'] 
Score: -14.556631088256836 	 ['covid-19', 'disease_treat_procedure', 'expression'] 
Score: -14.803701400756836 	 ['covid-19', 'disease_treat_procedure', 'detection'] 
Score: -15.057132720947266 	 ['covid-19', 'disease_treat_procedure', 'induction'] 
Score: -15.081883430480957 	 ['covid-19', 'disease_treat_procedure', 'reduction'] 
Score: -15.128449440002441 	 ['covid-19', 'disease_treat_procedure', 'infiltration'] 
Score: -15.141258239746094 	 ['covid-19', 'disease_treat_procedure', 'release'] 
Score: -15.15165901184082 	 ['covid-19', 'disease_treat_procedure', 'change'] 
Score: -14.057209014892578 	 ['presence', 'treat_procedur

TypeError: ignored

### Relations

### Query: what are the possible therapeutic procedure of covid-19??

In [ ]:
# Query: what are the possible therapeutic procedure of covid-19??

from ampligraph.discovery import query_topn

# restore the previously saved model to save time
model = restore_model('TransE.pkl')

triples, scores = query_topn(model, top_n=20, 
                             head='covid-19', 
                             relation='disease_treat_procedure', 
                             tail=None, 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

In [ ]:
# Query: what are the possible therapeutic procedure of covid-19??

from ampligraph.discovery import query_topn

# restore the previously saved model to save time
model = restore_model('TransE.pkl')

triples, scores = query_topn(model, top_n=20, 
                             head=None, 
                             relation='treat_procedure_disease', 
                             tail='covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

### Query: What are related diseases of Covid-19

In [ ]:
# Predict relation

triples, scores = query_topn(model, top_n=20, 
                             head='covid-19', 
                             relation='disease_disease', 
                             tail= None, 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

In [ ]:
# Predict relation

triples, scores = query_topn(model, top_n=20, 
                             head=None, 
                             relation='disease_disease', 
                             tail= 'covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

### Query: What are the genes/proteins related to Covid-19

In [ ]:
# Predict relation

triples, scores = query_topn(model, top_n=25, 
                             head='covid-19', 
                             relation='disease_gene', 
                             tail=None, 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

In [ ]:
# Predict relation

triples, scores = query_topn(model, top_n=25, 
                             head= None, 
                             relation='gene_disease', 
                             tail='covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

### Query: What species transmit the covid-19

In [ ]:
# Predict relation
triples, scores = query_topn(model, top_n=20, 
                             tail=None, 
                             relation='disease_species', 
                             head='covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

### Query: what are symptoms of the covid-19

In [ ]:

triples, scores = query_topn(model, top_n=25, 
                             head=None, 
                             relation='symptom_disease', 
                             tail='covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

In [ ]:
triples, scores = query_topn(model, top_n=25, 
                             tail=None, 
                             relation='disease_symptom', 
                             head='covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

### What are potential drugs to treat COVID-19?

In [ ]:

triples, scores = query_topn(model, top_n=30, 
                             head=None, 
                             relation='drug_disease', 
                             tail='covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

In [ ]:
# Compare the raw call and graph embedding results for the symptoms query

## raw call
#dataset.loc[(dataset['subject']=='symptom')].head(20)

#####==> the result of raw call seems to be better